In [1]:
import numpy as np
import pandas as pd
import random
from scipy.stats import dirichlet, expon, uniform, norm
from faker import Faker
fake = Faker()

In [2]:
# Define some global parameters
N_DAYS = 365*2
N_AUTHORS = 50
N_USERS = 1000

# Define possible topics and baseline probabilities // right now all made up, but should be based on data eventually
topics = ['Opinion', 'Politics', 'World Events', 'Business', 'Technology', 'Arts & Culture', 'Sports', 'Health', 'Home', 'Travel', 'Fashion', 'Food']
topics_probs = np.array([[0.1, 0.1, 0.1, 0.1, 0.08, 0.08, 0.08, 0.08, 0.08, 0.07, 0.07, 0.06]])

# Generate fake authors and probability vectors
authors_names = [fake.name() for i in range(N_AUTHORS)]
authors_quality = [uniform.rvs()*10 for i in range(N_AUTHORS)]

alphas = np.ones(len(topics)) # Concentration parameters TBD, right now uniform
authors_topicsprobs = np.array([dirichlet.rvs(alphas)[0] for auth in authors_names])
authors_popularity = dirichlet.rvs(np.ones(N_AUTHORS)*10)[0]

In [3]:
events = []
articles = []

# Iterate over days to generate all articles
for day in range(N_DAYS):

    # Generate new event with some probability (TBD)
    for _ in range(0, int(norm.rvs(loc=1, scale=2))):
        event_influence = dirichlet.rvs(alphas) # Concentration parameters TBD
        event_duration = expon.rvs(loc=0.01, scale=0.1) # some events should be long-lived
        duration_days = np.ceil(event_duration * 30)
        # hmm, event duration and intensity is generally correlated
        event_intensity = expon.rvs(scale=0.1, loc=0.1)
        events.append({'id': len(events), 'start': day, 'influence': event_influence, 'duration': duration_days, 'intensity': event_intensity})

    # Loop through events and sum topic probabilities of all active events
    day_topicsprobs = topics_probs.copy()
    for event in events:
        if event['start'] + event['duration'] >= day:
            day_topicsprobs += event['intensity']*event['influence']

    # Normalize probabilities
    day_topicsprobs = day_topicsprobs[0] / day_topicsprobs[0].sum()

    # Generate articles for day
    n_articles = round(norm.rvs(loc=N_AUTHORS / 3, scale=N_AUTHORS / 10))
    for i in range(n_articles):
        article_topic = np.random.choice(topics, p=day_topicsprobs)
        # Normalize probabilities
        author_probs = authors_topicsprobs[:, topics.index(article_topic)]
        author_probs = author_probs / author_probs.sum()
        article_author = np.random.choice(authors_names, p=author_probs)
        articles.append({'id': len(articles), 'day': day, 'topic': article_topic, 'author': article_author})

events_df = pd.DataFrame(events)
articles_df = pd.DataFrame(articles)

events_df.describe()

,id,start,duration,intensity
count,812.000000,812.000000,812.000000,812.000000
mean,405.500000,351.857143,3.935961,0.201609
std,234.548502,207.255257,3.093422,0.102752
min,0.000000,0.000000,1.000000,0.100036
25%,202.750000,170.750000,2.000000,0.130018
50%,405.500000,340.000000,3.000000,0.169464
75%,608.250000,517.000000,5.000000,0.236209
max,811.000000,728.000000,23.000000,0.785040


In [4]:
articles_df.describe()

,id,day
count,12269.000000,12269.000000
mean,6134.000000,365.227076
std,3541.899561,211.060908
min,0.000000,0.000000
25%,3067.000000,182.000000
50%,6134.000000,366.000000
75%,9201.000000,546.000000
max,12268.000000,729.000000


In [5]:
articles_df.groupby('topic').describe()

id                                                   \
                 count         mean          std   min      25%     50%   
topic                                                                     
Arts & Culture   983.0  6499.627670  3470.808903  29.0  3533.50  6788.0   
Business        1202.0  6092.211314  3479.471702   8.0  3181.25  5989.0   
Fashion          939.0  5978.133120  3614.679955   6.0  2740.00  5770.0   
Food             853.0  6153.900352  3521.053384   4.0  3254.00  5947.0   
Health          1008.0  6316.484127  3435.044801  24.0  3388.25  6571.5   
Home            1007.0  5964.553128  3510.350562  28.0  2959.50  5880.0   
Opinion         1183.0  6110.291631  3589.496256   0.0  3081.50  6050.0   
Politics        1129.0  6157.721878  3674.148749   7.0  2918.00  5984.0   
Sports           964.0  6169.717842  3487.510434   3.0  3144.25  6238.5   
Technology       935.0  6114.022460  3610.447911   1.0  2907.00  6062.0   
Travel           937.0  6200.529349  3510.515404  10.0  3084.00  6426.0   
World Events    1129.0  5894.907883  3556.194403  11.0  2789.00  5992.0   

                                     day                                       \
                    75%      max   count        mean         std  min     25%   
topic                                                                           
Arts & Culture  9444.50  12259.0   983.0  387.045778  206.716624  1.0  209.50   
Business        9112.75  12251.0  1202.0  362.764559  207.391420  0.0  189.00   
Fashion         9287.00  12234.0   939.0  355.972311  215.388050  0.0  164.00   
Food            9180.00  12235.0   853.0  366.508792  209.748718  0.0  193.00   
Health          9139.75  12245.0  1008.0  375.952381  204.757516  0.0  201.75   
Home            8813.50  12258.0  1007.0  355.103277  209.238716  1.0  176.50   
Opinion         9279.50  12261.0  1183.0  363.765850  213.920384  0.0  183.00   
Politics        9543.00  12254.0  1129.0  366.720106  218.968552  0.0  174.00   
Sports          9148.00  12255.0   964.0  367.330913  207.761869  0.0  186.00   
Technology      9318.50  12242.0   935.0  364.055615  215.155322  0.0  173.50   
Travel          9144.00  12266.0   937.0  369.160085  209.079239  0.0  183.00   
World Events    8858.00  12268.0  1129.0  350.982285  211.989712  0.0  167.00   

                                      
                  50%     75%    max  
topic                                 
Arts & Culture  407.0  562.00  728.0  
Business        356.0  540.75  728.0  
Fashion         344.0  553.00  726.0  
Food            354.0  545.00  726.0  
Health          392.0  542.25  727.0  
Home            351.0  521.00  728.0  
Opinion         361.0  552.50  729.0  
Politics        356.0  567.00  728.0  
Sports          371.0  542.25  728.0  
Technology      361.0  555.00  727.0  
Travel          383.0  542.00  729.0  
World Events    356.0  524.00  729.0

In [12]:
# Generate users
users = []
# will depend on how calculations are done

for user in range(N_USERS):
    ip = fake.ipv4() # not sure if these first two are necessary
    agent = fake.user_agent()
    prefs = dirichlet.rvs(alphas)
    freq = int(norm.rvs(loc=5, scale=5)) # should be bimodal, will investigate
    first_day = int(uniform.rvs()*N_DAYS)
    lifetime = int(expon.rvs(loc=20, scale=200))
    ad_sensitivity = norm.rvs(loc=2, scale=0.2)

    num_favorite = np.ceil(uniform.rvs()*4)
    favorites = random.sample(authors_names, int(num_favorite))
    users.append({'ip': ip, 'agent': agent, 'prefs': prefs, 'freq': max(freq, 0),
                  'first_day': first_day, 'lifetime': lifetime, 'ad_sensitivity': ad_sensitivity,
                  'favorites': favorites})

In [13]:
users_df = pd.DataFrame(users)
users_df

,ip,agent,prefs,freq,first_day,lifetime,ad_sensitivity,favorites
0,56.150.117.68,Opera/8.89.(Windows NT 5.01; mi-NZ) Presto/2.9...,"[[0.007360383115440668, 0.24262506824112237, 0...",9,406,336,1.890214,"[Amber Conrad, Anna Paul, Summer Cooper, Jeffr..."
1,200.138.183.159,Mozilla/5.0 (iPad; CPU iPad OS 5_1_1 like Mac ...,"[[0.0003219440359596212, 0.07805357963153772, ...",10,553,114,1.952494,"[Timothy Barber, John Leonard]"
2,191.141.230.178,Mozilla/5.0 (iPad; CPU iPad OS 9_3_5 like Mac ...,"[[0.3451409965398571, 0.07128343202032611, 0.0...",0,581,23,2.080753,"[Stephen Martin, Angela Quinn, Anna Paul]"
3,161.193.76.105,Mozilla/5.0 (iPod; U; CPU iPhone OS 3_3 like M...,"[[0.044746905311257336, 0.1292045243856901, 0....",0,356,622,2.077663,"[Steven Wilson, Michael Marks]"
4,213.25.71.98,Mozilla/5.0 (iPad; CPU iPad OS 14_2_1 like Mac...,"[[0.02838256503038247, 0.029219214813447275, 0...",6,260,104,2.009152,"[James Carter, Daniel Williams, Marcus Gilbert..."
...,...,...,...,...,...,...,...,...
995,159.87.121.16,Opera/9.41.(X11; Linux x86_64; aa-DJ) Presto/2...,"[[0.06443744160040796, 0.1409057062783758, 0.2...",2,286,198,1.841452,"[Donald Roberts, Cody Rowland]"
996,195.143.140.125,Mozilla/5.0 (X11; Linux i686; rv:1.9.6.20) Gec...,"[[0.10570553390363821, 0.01903586253419786, 0....",7,692,303,2.087367,"[Amber Conrad, Benjamin Becker]"
997,123.68.126.166,Opera/9.23.(X11; Linux i686; it-CH) Presto/2.9...,"[[0.14751608219080675, 0.16493720139565118, 0....",6,297,214,2.190526,[Sarah Marshall]
998,158.195.169.220,Mozilla/5.0 (Windows CE; ht-HT; rv:1.9.1.20) G...,"[[0.05847297968383385, 0.22396140008110005, 0....",2,462,301,1.827772,"[Katherine Murphy, Summer Cooper, Cynthia Step..."
